In [ ]:
## PCA IDEA

In [ ]:
# HYPEROPT USING ADJUSTED RAND INDEX
from hyperopt import hp, fmin, tpe, Trials
from sklearn.cluster import DBSCAN
from sklearn.metrics import adjusted_rand_score

# Define the search space
space = {
    'eps': hp.uniform('eps', 1, 5),
    'min_samples': hp.choice('min_samples', range(10, 31))
}

# Define the objective function
def objective(params):
    dbscan = DBSCAN(**params)
    cluster_labels = dbscan.fit_predict(tsne_results)

    # Identify non-noise points
    non_noise_indices = np.where(cluster_labels != -1)[0]
    filtered_labels = cluster_labels[non_noise_indices]
    filtered_true_labels = true_labels[non_noise_indices]

    score = adjusted_rand_score(filtered_true_labels, filtered_labels)
    return -score 
    # dbscan.fit(tsne_results)
    # score = adjusted_rand_score(true_labels, dbscan.labels_)
    # return -score  # Negative score because fmin tries to minimize the objective

# Run the optimizer
best = fmin(
    fn=objective,
    space=space,
    algo=tpe.suggest,
    max_evals=500  # Number of evaluations
)

best_min_samples = range(10, 31)[best['min_samples']]
best_eps = best['eps']
print("Best eps:", best['eps'])
print("Best min_samples:", best_min_samples)

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

# Normalizing the data
scaler = StandardScaler()
normalized_features = scaler.fit_transform(itemset_features)
print(len(normalized_features))

# Performing PCA
pca_3d = PCA(n_components=3)
pca_features_3d = pca_3d.fit_transform(normalized_features)

# Plotting the PCA transformed vectors in 3D
fig = plt.figure(figsize=(8, 6))
ax = fig.add_subplot(111, projection='3d')

for label in np.unique(true_labels):
    indices = np.where(true_labels == label)
    ax.scatter(pca_features_3d[indices, 0], pca_features_3d[indices, 1], pca_features_3d[indices, 2], label=label, s=1)

ax.set_xlabel('Principal Component 1')
ax.set_ylabel('Principal Component 2')
ax.set_zlabel('Principal Component 3')
ax.set_title('3D PCA of Transformed Vectors')
ax.legend()

plt.show()

In [ ]:
print(sum(pca_3d.explained_variance_ratio_))

In [ ]:
# function included with the sphericalkmeans library to evaluate the distance between the clusters
# dark means closer, light means further away - think ideally should have more light squares?

sphkmeans_centers = spherical_kmeans.cluster_centers_

fig = visualize_pairwise_distance(centers=sphkmeans_centers,
                                  labels=sphkmeans_cluster_labels, 
                                  max_dist=1, 
                                  sort=False)

In [ ]:
# extra

#agglomerative clustering

# from sklearn.metrics.pairwise import cosine_similarity
# Compute the cosine similarity matrix
# similarity = cosine_similarity(itemset_features)
# print(similarity.__sizeof__()/1024)


np.set_printoptions(threshold=3000)

print(true_labels)
print(sphkmeans_cluster_labels)

from sklearn.metrics import adjusted_rand_score

adjusted_rand_score(true_labels)


In [ ]:
# extra

#agglomerative clustering

# from sklearn.metrics.pairwise import cosine_similarity
# Compute the cosine similarity matrix
# similarity = cosine_similarity(itemset_features)
# print(similarity.__sizeof__()/1024)


np.set_printoptions(threshold=3000)

print(true_labels)
print(sphkmeans_cluster_labels)

from sklearn.metrics import adjusted_rand_score

adjusted_rand_score(true_labels, sphkmeans_cluster_labels)


In [ ]:
## RANDOM VISUALIZATION STUFF BELOW

In [ ]:
## Silhouette plot to visualize how good clusters are: if they are good, should see more on the right side
## Good average silhouette score apparently should be >0.5?

visualize_silhouette_clusters(sphkmeans_cluster_labels, itemset_features, metric='cosine')

In [ ]:
## SPH KMEANS MORE IDEA

In [ ]:
# adjust number of clusters
increased_num_clusters = 3

# convert itemset_features to a sparse matrix that SphericalKMeans will accept
itemset_matrix = csr_matrix(itemset_features.astype(int))
density = itemset_matrix.getnnz() / np.prod(itemset_matrix.shape)
print("density of matrix:", density)

spherical_kmeans = SphericalKMeans(
        n_clusters=increased_num_clusters,
        max_iter=100,
        verbose=1,
        init='similar_cut',
        #sparsity='minimum_df',
        minimum_df_factor=0.05
    )

increased_sphkmeans_cluster_labels = spherical_kmeans.fit_predict(itemset_matrix)
print('finished spherical clustering')

compare_true_labels(true_labels, increased_sphkmeans_cluster_labels, itemset_features)

In [ ]:
plt.figure(figsize=(10, 6))
plt.scatter(tsne_results[:, 0], tsne_results[:, 1], c=increased_sphkmeans_cluster_labels, cmap='viridis')
plt.colorbar()
plt.title('t-SNE Visualization of Clustering Results')
plt.xlabel('t-SNE Dimension 1')
plt.ylabel('t-SNE Dimension 2')
plt.show()

In [ ]:
# BISECTING SPHERICAL K MEANS

from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse import csr_matrix
from sklearn.cluster import KMeans
import numpy as np

# Function to compute intra-cluster variance
def intra_cluster_variance(cluster_data):
    if len(cluster_data) < 2:
        return 0
    centroid = np.mean(cluster_data, axis=0)
    variance = np.mean(np.square(cosine_similarity(cluster_data, centroid.reshape(1, -1)) - 1))
    return variance

# Initialization
n_clusters = increased_num_clusters
clusters = [csr_matrix(itemset_features)]
final_labels = np.zeros(len(itemset_features), dtype=int)

# Bisecting step
while len(clusters) < n_clusters:
    highest_variance = 0
    cluster_to_split = None
    cluster_index = -1

    # Find the cluster with the highest intra-cluster variance
    for i, cluster in enumerate(clusters):
        variance = intra_cluster_variance(cluster.toarray())
        if variance > highest_variance:
            highest_variance = variance
            cluster_to_split = cluster
            cluster_index = i

    # Perform spherical k-means on the cluster with the highest variance
    spherical_kmeans = SphericalKMeans(
        n_clusters=2,
        max_iter=100,
        verbose=1,
        init='similar_cut',
        minimum_df_factor=0.05
    )

    sub_cluster_labels = spherical_kmeans.fit_predict(cluster_to_split)

    # Replace the original cluster with the two new sub-clusters
    clusters.pop(cluster_index)
    for new_cluster_index in [0, 1]:
        new_cluster_data = cluster_to_split.toarray()[sub_cluster_labels == new_cluster_index, :]
        clusters.append(csr_matrix(new_cluster_data))

    # Update labels
    original_indices = np.where(final_labels == cluster_index)[0]
    final_labels[original_indices[sub_cluster_labels == 1]] = len(clusters) - 1
    final_labels[original_indices[sub_cluster_labels == 0]] = cluster_index

# At this point, `labels` contains the cluster assignments for each data point

compare_true_labels(true_labels, final_labels, itemset_features)
print(np.unique(final_labels))

plt.figure(figsize=(10, 6))
plt.scatter(tsne_results[:, 0], tsne_results[:, 1], c=final_labels, cmap='viridis')
plt.colorbar()
plt.title('t-SNE Visualization of Clustering Results')
plt.xlabel('t-SNE Dimension 1')
plt.ylabel('t-SNE Dimension 2')
plt.show()

In [ ]:
# KModes clustering

km = KModes(n_clusters=increased_num_clusters, init='Huang', n_init=5, verbose=1, n_jobs=-1)
kmodes_cluster_labels = km.fit_predict(itemset_features)

compare_true_labels(true_labels, kmodes_cluster_labels, itemset_features)

In [ ]:
plt.figure(figsize=(10, 6))
plt.scatter(tsne_results[:, 0], tsne_results[:, 1], c=kmodes_cluster_labels, cmap='viridis')
plt.colorbar()
plt.title('t-SNE Visualization of Clustering Results')
plt.xlabel('t-SNE Dimension 1')
plt.ylabel('t-SNE Dimension 2')
plt.show()

In [ ]:
distances = cosine_distances(itemset_features)

agglo_clustering = AgglomerativeClustering(n_clusters=increased_num_clusters, metric="precomputed", linkage="average")
agglo_labels = agglo_clustering.fit_predict(distances)

compare_true_labels(true_labels, agglo_labels, itemset_features)